In [1]:
import psycopg2
import pandas as pd
import numpy as np
import warnings
import seaborn as sns
from matplotlib import pyplot as plt
warnings.simplefilter("ignore", UserWarning)

In [2]:
#pip install python-dotenv

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
dbname = os.getenv("dbname")
host = os.getenv("host")
port = os.getenv("port")
user = os.getenv("user")
password = os.getenv("password")

In [4]:
print(user)

natalia


In [5]:
def connect_to_redshift(dbname, host, port, user, password):
    """Method that connects to redshift. This gives a warning so will look for another solution"""

    connect = psycopg2.connect(
        dbname=dbname, host=host, port=port, user=user, password=password
    )

    print("connection to redshift made")

    return connect

In [6]:
connect=connect_to_redshift(dbname, host, port, user, password)

connection to redshift made


In [7]:
query="""select otc.*
              
         from bootcamp.online_transactions_cleaned otc   
      
      """

In [8]:
otc=pd.read_sql(query,connect)

In [9]:
otc.head(10)

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536384,22189,CREAM HEART CARD HOLDER,3.95,4,15.80,2010-12-01 09:53:00,u18074,United Kingdom
1,536392,21891,TRADITIONAL WOODEN SKIPPING ROPE,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
2,536401,22068,BLACK PIRATE TREASURE CHEST,1.65,2,3.30,2010-12-01 11:21:00,u15862,United Kingdom
3,536412,22961,JAM MAKING SET PRINTED,1.45,24,34.80,2010-12-01 11:49:00,u1792,United Kingdom
4,536420,21889,WOODEN BOX OF DOMINOES,1.25,12,15.00,2010-12-01 12:03:00,u16583,United Kingdom
5,536464,22742,MAKE YOUR OWN PLAYTIME CARD KIT,2.95,1,2.95,2010-12-01 12:23:00,u17968,United Kingdom
6,536520,22557,PLASTERS IN TIN VINTAGE PAISLEY,1.65,2,3.30,2010-12-01 12:43:00,u14729,United Kingdom
7,536525,20973,12 PENCIL SMALL TUBE WOODLAND,0.65,2,1.30,2010-12-01 12:54:00,u14078,United Kingdom
8,536378,84519A,TOMATO CHARLIE+LOLA COASTER SET,2.95,6,17.70,2010-12-01 09:37:00,u14688,United Kingdom
9,536401,22100,SKULLS SQUARE TISSUE BOX,1.25,2,2.50,2010-12-01 11:21:00,u15862,United Kingdom


In [10]:
otc.shape

(399841, 9)



## From the initial otc_cleaned table with 399841 records the following records were dropped:

1. 4026 records cancellation pairs 
2. 34 records with zero prices
3. 153 records with invoices to beginn with "C","D","P" letters. The particular invoices belong probably to expenses.

The new otc_cleaned has 395628 records.

## 1. Negative quantities (Finding cancellation pairs)
8507 records with negative quantities


In [13]:
otc_negative=otc[otc.total_order_value<0]
otc_negative

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
60,C536548,22580,ADVENT CALENDAR GINGHAM SACK,5.95,-4,-23.80,2010-12-01 14:33:00,u12472,Germany
271,C536820,22878,Unknown,2.10,-1,-2.10,2010-12-02 17:14:00,u18168,United Kingdom
282,C536548,21218,RED SPOTTY BISCUIT TIN,3.75,-3,-11.25,2010-12-01 14:33:00,u12472,Germany
337,C536850,22942,CHRISTMAS LIGHTS 10 SANTAS,7.65,-48,-367.20,2010-12-03 09:47:00,u14213,United Kingdom
384,C536807,22501,PICNIC BASKET WICKER LARGE,9.95,-2,-19.90,2010-12-02 16:45:00,u15834,United Kingdom
...,...,...,...,...,...,...,...,...,...
399665,C580968,23583,LUNCH BAG PAISLEY PARK,1.65,-10,-16.50,2011-12-06 14:57:00,u15622,United Kingdom
399667,C580989,22331,WOODLAND PARTY BAG + STICKER SET,1.65,-1,-1.65,2011-12-06 16:42:00,u12709,Germany
399689,C581145,23249,ENAMEL DINNER PLATE PANTRY,1.49,-9,-13.41,2011-12-07 13:48:00,u1749,United Kingdom
399765,C581322,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,0.85,-1,-0.85,2011-12-08 11:51:00,u18223,United Kingdom


In [14]:
c_pairs=otc.groupby([otc.customer_id, otc.stock_code, otc.quantity.abs()])\
            .filter(lambda x: (len(x.quantity.abs()) % 2 == 0) and (x.quantity.sum() == 0))\
            .sort_values(by=['customer_id', 'stock_code'])

From 8507 records with negative quantities 4026 are cancellation pairs. The particular transactions will not be included, as they correspond to cancellation of orders or returns.

In [15]:
c_pairs

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
108632,C541433,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,-74215,-77183.6,2011-01-18 10:17:00,u12346,United Kingdom
111875,541431,23166,MEDIUM CERAMIC TOP STORAGE JAR,1.04,74215,77183.6,2011-01-18 10:01:00,u12346,United Kingdom
112024,543370,22839,3 TIER CAKE TIN GREEN AND CREAM,14.95,2,29.9,2011-02-07 14:51:00,u12359,Cyprus
149653,C549955,22839,3 TIER CAKE TIN GREEN AND CREAM,14.95,-2,-29.9,2011-04-13 13:38:00,u12359,Cyprus
373165,573173,22941,CHRISTMAS LIGHTS 10 REINDEER,8.50,2,17.0,2011-10-28 10:10:00,u12362,Belgium
...,...,...,...,...,...,...,...,...,...
273974,C577832,23245,SET OF 3 REGENCY CAKE TINS,4.95,-4,-19.8,2011-11-22 10:18:00,u18274,United Kingdom
264651,575485,84509A,SET OF 4 ENGLISH ROSE PLACEMATS,3.75,4,15.0,2011-11-09 17:03:00,u18274,United Kingdom
340472,C577832,84509A,SET OF 4 ENGLISH ROSE PLACEMATS,3.75,-4,-15.0,2011-11-22 10:18:00,u18274,United Kingdom
287749,C577832,84988,SET OF 72 PINK HEART PAPER DOILIES,1.45,-12,-17.4,2011-11-22 10:18:00,u18274,United Kingdom


In [19]:
c_pairs_df=c_pairs.index.to_frame().reset_index()
c_pairs_df

,index,0
0,108632,108632
1,111875,111875
2,112024,112024
3,149653,149653
4,373165,373165
...,...,...
4021,273974,273974
4022,264651,264651
4023,340472,340472
4024,287749,287749


In [20]:
c_pairs_df = c_pairs_df.drop(columns=c_pairs_df.columns[1])
c_pairs_df

,index
0,108632
1,111875
2,112024
3,149653
4,373165
...,...
4021,273974
4022,264651
4023,340472
4024,287749


In [21]:
otc_final = otc.drop(labels=c_pairs.index, axis=0)

From the initial otc table with 399841 records the 4026 were dropped. The new table has 395815 records.

In [22]:
otc_final

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536384,22189,CREAM HEART CARD HOLDER,3.95,4,15.80,2010-12-01 09:53:00,u18074,United Kingdom
1,536392,21891,TRADITIONAL WOODEN SKIPPING ROPE,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
2,536401,22068,BLACK PIRATE TREASURE CHEST,1.65,2,3.30,2010-12-01 11:21:00,u15862,United Kingdom
3,536412,22961,JAM MAKING SET PRINTED,1.45,24,34.80,2010-12-01 11:49:00,u1792,United Kingdom
4,536420,21889,WOODEN BOX OF DOMINOES,1.25,12,15.00,2010-12-01 12:03:00,u16583,United Kingdom
...,...,...,...,...,...,...,...,...,...
399836,581476,21479,WHITE SKULL HOT WATER BOTTLE,4.25,4,17.00,2011-12-09 08:48:00,u12433,Norway
399837,581481,23178,JAM CLOCK MAGNET,1.25,12,15.00,2011-12-09 09:07:00,u1749,United Kingdom
399838,581496,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,0.85,12,10.20,2011-12-09 10:20:00,u16558,United Kingdom
399839,581516,21620,SET OF 4 ROSE BOTANICAL CANDLES,1.25,12,15.00,2011-12-09 11:26:00,u14422,United Kingdom


In [69]:
#399841-4026=395815


## 2. Zero prices will be removed from the dataset.
There are 34 records with zero prices

In [23]:
otc_zero_price=otc_final[otc.price==0]
otc_zero_price

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
2756,537197,22841,ROUND CAKE TIN VINTAGE GREEN,0.0,1,0.0,2010-12-05 14:02:00,u12647,Germany
11789,540372,22553,PLASTERS IN TIN SKULLS,0.0,24,0.0,2011-01-06 16:41:00,u13081,United Kingdom
18624,540372,22090,PAPER BUNTING RETROSPOT,0.0,24,0.0,2011-01-06 16:41:00,u13081,United Kingdom
24856,539722,22423,REGENCY CAKESTAND 3 TIER,0.0,10,0.0,2010-12-21 13:45:00,u14911,EIRE
32301,550188,22636,CHILDS BREAKFAST SET CIRCUS PARADE,0.0,1,0.0,2011-04-14 18:57:00,u12457,Switzerland
55602,543599,84535B,FAIRY CAKES NOTEBOOK A6 SIZE,0.0,16,0.0,2011-02-10 13:08:00,u1756,United Kingdom
67516,548871,22162,HEART GARLAND RUSTIC PADDED,0.0,2,0.0,2011-04-04 14:42:00,u1441,United Kingdom
77679,548318,22055,MINI CAKE STAND HANGING STRAWBERY,0.0,5,0.0,2011-03-30 12:45:00,u13113,United Kingdom
88757,553000,47566,PARTY BUNTING,0.0,4,0.0,2011-05-12 15:21:00,u17667,United Kingdom
89070,539263,22580,ADVENT CALENDAR GINGHAM SACK,0.0,4,0.0,2010-12-16 14:36:00,u1656,United Kingdom


In [24]:
otc_zero_price.shape


(34, 9)

In [25]:
otc_zero_price.index

Index([  2756,  11789,  18624,  24856,  32301,  55602,  67516,  77679,  88757,
        89070, 104084, 120800, 152981, 159062, 164326, 205836, 214254, 214369,
       227548, 235877, 239790, 250113, 266874, 292898, 299778, 304951, 321428,
       341948, 355164, 357987, 369182, 379000, 390856, 395826],
      dtype='int64')

In [27]:
otc_final=otc_final.drop(labels=otc_zero_price.index,axis=0)

In [28]:
otc_final.shape


(395781, 9)

## 3. Invoices with "C","D","P" will be not included, as they correspond to the expenses or costs of the company

In [29]:
CDP_stock_code=otc_final[otc_cleaned_final.stock_code.str[0].isin(["C","D","P"])]
CDP_stock_code

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
5327,536540,C2,CARRIAGE,50.00,1,50.00,2010-12-01 14:05:00,u14911,EIRE
8276,537378,C2,CARRIAGE,50.00,1,50.00,2010-12-06 13:06:00,u14911,EIRE
15619,540547,C2,CARRIAGE,50.00,1,50.00,2011-01-09 16:07:00,u14911,EIRE
18555,538002,C2,CARRIAGE,50.00,1,50.00,2010-12-09 11:48:00,u14932,Channel Islands
18764,539421,C2,CARRIAGE,50.00,1,50.00,2010-12-17 14:21:00,u14016,EIRE
...,...,...,...,...,...,...,...,...,...
363114,565383,DOT,DOTCOM POSTAGE,16.46,1,16.46,2011-09-02 15:45:00,u14096,United Kingdom
367611,569246,DOT,DOTCOM POSTAGE,490.25,1,490.25,2011-10-03 09:57:00,u14096,United Kingdom
375479,571441,DOT,DOTCOM POSTAGE,743.62,1,743.62,2011-10-17 13:31:00,u14096,United Kingdom
391525,575607,DOT,DOTCOM POSTAGE,908.16,1,908.16,2011-11-10 12:37:00,u14096,United Kingdom


In [30]:
CDP_stock_code.index

Index([  5327,   8276,  15619,  18555,  18764,  21979,  24018,  24818,  29216,
        30973,
       ...
       328894, 328983, 343339, 348114, 354181, 363114, 367611, 375479, 391525,
       393418],
      dtype='int64', length=153)

In [31]:
otc_final=otc_final.drop(labels=CDP_stock_code.index,axis=0)
otc_final

,invoice,stock_code,description,price,quantity,total_order_value,invoice_date,customer_id,country
0,536384,22189,CREAM HEART CARD HOLDER,3.95,4,15.80,2010-12-01 09:53:00,u18074,United Kingdom
1,536392,21891,TRADITIONAL WOODEN SKIPPING ROPE,1.25,12,15.00,2010-12-01 10:29:00,u13705,United Kingdom
2,536401,22068,BLACK PIRATE TREASURE CHEST,1.65,2,3.30,2010-12-01 11:21:00,u15862,United Kingdom
3,536412,22961,JAM MAKING SET PRINTED,1.45,24,34.80,2010-12-01 11:49:00,u1792,United Kingdom
4,536420,21889,WOODEN BOX OF DOMINOES,1.25,12,15.00,2010-12-01 12:03:00,u16583,United Kingdom
...,...,...,...,...,...,...,...,...,...
399836,581476,21479,WHITE SKULL HOT WATER BOTTLE,4.25,4,17.00,2011-12-09 08:48:00,u12433,Norway
399837,581481,23178,JAM CLOCK MAGNET,1.25,12,15.00,2011-12-09 09:07:00,u1749,United Kingdom
399838,581496,22909,SET OF 20 VINTAGE CHRISTMAS NAPKINS,0.85,12,10.20,2011-12-09 10:20:00,u16558,United Kingdom
399839,581516,21620,SET OF 4 ROSE BOTANICAL CANDLES,1.25,12,15.00,2011-12-09 11:26:00,u14422,United Kingdom


In [124]:
# 395781-153=395628


## Load 'otc_final' to csv file for the customer segmentation analysis

In [32]:
otc_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 395628 entries, 0 to 399840
Data columns (total 9 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   invoice            395628 non-null  object        
 1   stock_code         395628 non-null  object        
 2   description        395628 non-null  object        
 3   price              395628 non-null  float64       
 4   quantity           395628 non-null  int64         
 5   total_order_value  395628 non-null  float64       
 6   invoice_date       395628 non-null  datetime64[ns]
 7   customer_id        395628 non-null  object        
 8   country            395628 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 30.2+ MB


In [39]:
otc_final.to_csv("./data/otc_final.csv",index=False)